In [3]:
import pandas as pd
import sqlite3

In [1]:


transactions = pd.read_csv("../walkthrough-data/unclean_transactions.csv")

# 1. How many transactions are there in total?

transactions_count = transactions.shape[0]

print(f"transactions count: {transactions_count}")

# 2. How many duplicates exist in the dataset?

duplicates = transactions.duplicated().sum()

print(f"transactions duplicates count: {duplicates}")

customers = pd.read_csv("../walkthrough-data/unclean_customers.csv")

# 3. How many customers are there in total?

customers_count = customers.shape[0]

print(f"customers count: {customers_count}")

# 4. How many duplicates exist in the dataset?

duplicates = customers.duplicated().sum()

print(f"customer duplicates count: {duplicates}")

# merge customers and transactions on customer_id

merged = pd.merge(customers, transactions, on="customer_id")

# How many duplicates exist in the merged dataset?

duplicates = merged.duplicated().sum()

print(f"merged duplicates: {duplicates}")


transactions count: 10500
transactions duplicates count: 2468
customers count: 5030
customer duplicates count: 0
merged duplicates: 2369


In [ ]:
# randomly remove 4.93% of transactions to simulate data loss

transactions = transactions.sample(frac=0.9507)

# how many transactions now?

transactions_count = transactions.shape[0]

print(transactions_count)

# Randomly duplicate 518 rows to make count 10500 again

transactions = pd.concat([transactions, transactions.sample(n=518)])

# how many transactions now?

transactions_count = transactions.shape[0]

print(transactions_count)

# write this back to the original file

# transactions.to_csv("../walkthrough-data/unclean_transactions.csv", index=False)

In [ ]:
# randomly remove 3.26% of customers to simulate data loss

customers = customers.sample(frac=0.9674)

# how many customers now?

customers_count = customers.shape[0]

print(customers_count)

# Randomly duplicate 170 rows to make count 5200 again

customers = pd.concat([customers, customers.sample(n=170)])

# how many customers now?

customers_count = customers.shape[0]

print(customers_count)

# write this back to the original file

# customers.to_csv("../walkthrough-data/unclean_customers.csv", index=False)


In [ ]:
# merge customers and transactions on customer_id

merged = pd.merge(customers, transactions, on="customer_id")

# How many duplicates exist in the merged dataset?

duplicates = merged.duplicated().sum()

print(duplicates)


In [ ]:
# How many rows in transactions have amount set to the string "INVALID"?

invalid_amounts = transactions[transactions["amount"] == "INVALID"].shape[0]

print(invalid_amounts)

# Set 0.3% of the amounts to "INVALID" to simulate data corruption

transactions.loc[transactions.sample(frac=0.003).index, "amount"] = "INVALID"

# How many rows in transactions have amount set to the string "INVALID" now?

invalid_amounts = transactions[transactions["amount"] == "INVALID"].shape[0]

print(invalid_amounts)

# How many rows are will be dropped if we removed nulls in this column?

nulls = transactions["amount"].isnull().sum()

print(nulls)


In [ ]:
#write back to csv files

transactions.to_csv("../walkthrough-data/unclean_transactions.csv", index=False)
customers.to_csv("../walkthrough-data/unclean_customers.csv", index=False)

In [27]:
# Save to SQLite


conn = sqlite3.connect("../walkthrough-data/unclean_transactions.db")
transactions.to_sql("transactions", conn, if_exists="replace", index=False)

# Generate SQL file
with open("../walkthrough-data/unclean_transactions.sql", "w") as sql_file:
    for line in conn.iterdump():
        sql_file.write(f"{line}\n")

# Close the connection
conn.close()

In [20]:
# Create in mem db from .db file and import table

conn = sqlite3.connect(":memory:")

with open("../walkthrough-data/old_unclean_transactions.sql") as sql_file:
    conn.executescript(sql_file.read())
    
# How many rows are in the transactions table?

old_transactions_count = pd.read_sql("SELECT COUNT(*) FROM transactions", conn)

print(old_transactions_count)

old_transactions = pd.read_sql("SELECT * FROM transactions", conn)

# duplicates?

duplicates = old_transactions.duplicated().sum()

print(duplicates)

#drop duplicated

old_transactions.drop_duplicates(inplace=True)

# how many rows now?

old_transactions_count = old_transactions.shape[0]

print(f"old_transactions_count: {old_transactions_count}")

# How many with transaction id over 10500?

over_10500 = old_transactions[old_transactions["transaction_id"] > 10500].shape[0]

print(over_10500)

# How many duplicates of transaction_id?

duplicates = old_transactions["transaction_id"].duplicated().sum()

print(duplicates)

# drop these duplicates

old_transactions.drop_duplicates(subset=["transaction_id"], inplace=True)

# how many rows now?

old_transactions_count = old_transactions.shape[0]

print(f"old_transactions_count: {old_transactions_count}")

# write to a temp csv file

old_transactions.to_csv("../walkthrough-data/starter_unclean_transactions.csv", index=False)

   COUNT(*)
0     11052
92
old_transactions_count: 10960
0
460
old_transactions_count: 10500


In [21]:
# randomly remove 4.93% of transactions to simulate data loss

old_transactions = old_transactions.sample(frac=0.9507)

# how many old_transactions now?

old_transactions_count = old_transactions.shape[0]

print(old_transactions_count)

# Randomly duplicate 518 rows to make count 10500 again

old_transactions = pd.concat([old_transactions, old_transactions.sample(n=518)])

# how many old_transactions now?

old_transactions_count = old_transactions.shape[0]

print(old_transactions_count)

old_transactions.to_csv("../walkthrough-data/duplicates_in_unclean_transactions.csv", index=False)


9982
10500


In [22]:
# How many with invalid amount?

invalid_amounts = old_transactions[old_transactions["amount"] == "INVALID"].shape[0]

print(invalid_amounts)



0


In [23]:
# randomly add 0.36% of invalid amounts

old_transactions.loc[old_transactions.sample(frac=0.0036).index, "amount"] = "INVALID"

# How many with invalid amount now?

invalid_amounts = old_transactions[old_transactions["amount"] == "INVALID"].shape[0]

print(invalid_amounts)



41


/var/folders/jf/3t017l9948jgt5frxgh1sqjm0000gn/T/ipykernel_53474/1619114310.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'INVALID' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  old_transactions.loc[old_transactions.sample(frac=0.0036).index, "amount"] = "INVALID"


In [24]:
# how many with other nulls or Nones?

nulls = old_transactions["amount"].isnull().sum()

print(nulls)

46


In [31]:
# Set of checks
old_transactions_duplicates = old_transactions.duplicated().sum()

print(f"old_transactions duplicated: {old_transactions_duplicates}")

old_transactions duplicated 518


In [32]:
# nulls and invalids to remove

old_transactions_nulls_invalids = old_transactions[old_transactions["amount"].isnull() | (old_transactions["amount"] == "INVALID")].shape[0]

print(f"old_transactions nulls and invalids: {old_transactions_nulls_invalids}")

old_transactions nulls and invalids: 87


In [33]:
# How many duplicated rows have nulls or invalids?

duplicates = old_transactions[old_transactions.duplicated() & (old_transactions["amount"].isnull() | (old_transactions["amount"] == "INVALID"))].shape[0]

print(f"duplicates with nulls or invalids: {duplicates}")


duplicates with nulls or invalids: 3


In [34]:
# Write unclean data

old_transactions.to_csv("../walkthrough-data/unclean_transactions.csv", index=False)

In [42]:
# Removed duplicates, nulls and invalids

check_transactions = pd.read_csv("../walkthrough-data/unclean_transactions.csv")

# How many duplicates to remove

duplicates = check_transactions.duplicated().sum()

print(f"duplicates: {duplicates}")

# How many missing to remove?

nulls = check_transactions["amount"].isnull().sum()

print(f"nulls: {nulls}")

# How many invalids to remove?

invalids = check_transactions[check_transactions["amount"] == "INVALID"].shape[0]

print(f"invalids: {invalids}")

# How many missing dates?

missing_dates = check_transactions["transaction_date"].isnull().sum()

print(f"missing_dates: {missing_dates}")

# How many duplicates have missing or invalid data?

duplicates = check_transactions[check_transactions.duplicated() & (check_transactions["amount"].isnull() | (check_transactions["amount"] == "INVALID") | check_transactions["transaction_date"].isnull())].shape[0]

print(f"duplicates with missing or invalid data: {duplicates}")

# remove duplicates, nulls and invalids

check_transactions.drop_duplicates(inplace=True)
check_transactions.dropna(subset=["amount"], inplace=True)
check_transactions.dropna(subset=["transaction_date"], inplace=True)
check_transactions = check_transactions[check_transactions["amount"] != "INVALID"]

# report how many rows are left

check_transactions_count = check_transactions.shape[0]

print(f"check_transactions_count: {check_transactions_count}")

duplicates: 518
nulls: 46
invalids: 41
missing_dates: 208
duplicates with missing or invalid data: 15
check_transactions_count: 9702


In [44]:
# create in mem database from unclean_transactions.csv

conn = sqlite3.connect(":memory:")

unclean_transactions = pd.read_csv("../walkthrough-data/unclean_transactions.csv")

unclean_transactions.to_sql("transactions", conn, if_exists="replace", index=False)

# How many rows are in the transactions table?

unclean_transactions_count = pd.read_sql("SELECT COUNT(*) FROM transactions", conn)

print(unclean_transactions_count)

   COUNT(*)
0     10500


In [45]:
#Create a .db file and an SQL file from the in mem db

with open("../walkthrough-data/new_unclean_transactions.sql", "w") as sql_file:
    for line in conn.iterdump():
        sql_file.write(f"{line}\n")

conn.close()

# create a .db file from the unclean_transactions.csv
